In [1]:
from os.path import join as pjoin
import os
import numpy as np
from nilearn.image import load_img, new_img_like
from nilearn.masking import apply_mask, unmask
from tqdm import tqdm
from dimension_encoding.b5k import B5kLoader

In [2]:
dl = B5kLoader()

B5k data loading from: ../data/b5k/


In [3]:
# responses = dl.load_responses("CSI1")